In [2]:
import numpy as np
import matplotlib.pyplot as plt
g=9.81

In [3]:
class X():
    def __init__(self,state=np.zeros(6)):
        self.x,self.y,self.phi=state[0],state[1],state[2]
        self.xDot,self.yDot,self.phiDot=state[3],state[4],state[5]
    def __call__(self):
        return [self.x,self.y,self.phi,self.xDot,self.yDot,self.phiDot]




In [4]:
class Obstacle:
    def __init__(self,x,y,r):
        self.c=np.array([x,y])
        self.r=r
    def CheckCollision(self,state):
        nChecks=state.shape[1]
        for i in range(nChecks):
            currentState=X(state[:,i])
            distance=np.linalg.norm(np.array([currentState.x,currentState.y]-self.c))
            if distance<=self.r:
                return 1
        return 0
    def __call__(self):
        return self.c,self.r

In [5]:
from scipy.integrate import solve_ivp
goalWidth=20
XLim=800
floor=0
cieling=450


class Quadcopter:
    def __init__(self,m,L,J,state=np.zeros(6)):
        self.M=0    #Moment (ie rotation)
        self.L=L
        self.mass=m
        self.torque=J
        self.F=self.mass*g    #Forward Thrust, EQ point
        self.state=X(state)
        self.diffState=None
    def dynamics(self,inputs,state=None):
        if state is not None:
            self.state=X(state)
        self.F,self.M=inputs
        xDDot=-self.F/self.mass*np.sin(self.state.phi)
        yDDot=self.F/self.mass*np.cos(self.state.phi)-g
        phiDDot=self.M/self.torque
        self.diffState=X([self.state.xDot,self.state.yDot,self.state.phiDot,xDDot,yDDot,phiDDot])
        return self.diffState()
    def solve(self,inputs,nSim=[0,15],nChecks=1000):
        nEval = np.linspace(nSim[0],nSim[1],nChecks)
        solution=solve_ivp(fun=lambda t, state: self.dynamics(inputs,state),
                           t_span=nSim,
                           t_eval=nEval,
                           y0=self.state()
                           )
        self.state= X(solution.y[:, -1])
        return solution.y
    
    
    

In [6]:
class SimulationEnviroment:
    def __init__(self,goal=[250,200]):
        self.obstacle=list()
        self.goal=Obstacle(goal[0],goal[1],goalWidth)
    def CreateObstacle(self,x,y,r):
        self.obstacle.append(Obstacle(x,y,r))
    def checkSolution(self,solution):
        if (solution[1]<floor).any():
            print("Crashed into the ground")
            #return None
        elif (abs(solution[0])>XLim).any():
            print("Crashed into the walls")
        elif (solution[0]>cieling):
            print("Crashed into the cieling")
        for obstacle in self.obstacle:
            if obstacle.CheckCollision(solution):
                print("Crashed into an obstacle")
                #return None
        if self.goal.CheckCollision(solution):
            print("Reached Goal")
    def PlotObjects(self,ax):
        for obstacle in self.obstacle:
            c,r=obstacle()
            circle=plt.Circle((c[0],c[1]),r,color="red",alpha=0.7)
            ax.add_patch(circle)
        c,r=self.goal()   
        circle=plt.Circle((c[0],c[1]),r,color="green",alpha=0.7)
        ax.add_patch(circle)

In [7]:
#Quadcopter and base simulation
m=1; L=1; J=1 #Kg,m,Kgm^2
Quad=Quadcopter(m,L,J)
Env=SimulationEnviroment(goal=[700,350])
Env.CreateObstacle(100,250,100)
Env.CreateObstacle(400,150,50)
Env.CreateObstacle(500,350,50)
Env.CreateObstacle(700,200,75)
sol=Quad.solve([4*m*g,-.2],nChecks=1000) #Inputs, and number of checks
Env.checkSolution(sol)

Crashed into the walls
Crashed into an obstacle
Crashed into an obstacle
Crashed into an obstacle
